In [1]:
import pandas as pd
import numpy as np
import ast
import re
from zhon.hanzi import non_stops , stops
import os
import pickle
from opencc import OpenCC
# from transformers import RobertaForTokenClassification
import torch
import torch.nn.functional as F
import pandas as pd
from transformers import RobertaTokenizer , RobertaForSequenceClassification
from transformers import BertTokenizer , BertConfig , BertModel, XLNetTokenizer ,XLNetModel
import numpy as np
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from transformers import BertForSequenceClassification,BertForPreTraining
from torch.autograd import Variable
from transformers import BertForPreTraining
from torch import nn
import json
import requests

In [48]:


dataset = 1
path = './dataset/dataset' + str(dataset) + '/'

train_df = pd.read_csv(path + "tbrain_train.csv")
train_df = train_df.fillna('None')

test_df = pd.read_csv(path + "tbrain_test.csv")
test_df = test_df.fillna('None')


lm_path = './bert_wwm_pretrain_tbrain/'

print(train_df.shape)
print(test_df.shape)

(4426, 4)
(491, 4)


In [3]:

def clean_string(content):
#     cc = OpenCC('t2s')
    content = content.replace('\n','。').replace('\t','，').replace('!', '！').replace('?', '？')# erease white space cause English name error
    content = re.sub("[+\.\/_,$%●▼►^*(+\"\']+|[+——~@#￥%……&*（）★]", "",content)
    content = re.sub(r"[%s]+" %stops, "。",content)
#     content = cc.convert(content)
    return content

def cut_sent(para):
    para = re.sub('([。！？\?])([^”’])', r"\1\n\2", para)
    para = re.sub('(\.{6})([^”’])', r"\1\n\2", para) 
    para = re.sub('(\…{2})([^”’])', r"\1\n\2", para)  
    para = re.sub('([。！？\?][”’])([^，。！？\?])', r'\1\n\2', para)
    return para.split("\n")


def combine_sentence(sentences , max_len):
    li = []
    string = ''
    for k in range(len(sentences)):
        sentence = sentences[k]
        if len(string) + len(sentence) < max_len:
            string = string + sentence
        else:
#             原本是空的代表sentences太長
            if string == '':
                n = max_len
                tmp_li = [sentence[i:i+n] for i in range(0, len(sentence), n)]
                string = tmp_li.pop(-1)
                li = li + tmp_li
            else:
                li.append(string)
                string = sentence
    if(string != ''):
        li.append(string)
    return li
        
    
    
    

In [4]:
def get_model_input_split(df):
    tokenizer = BertTokenizer.from_pretrained(lm_path)
    max_seq_length = 512 #xlnet no limit try try
    train_x = [] #all train_x
    train_input_ids = []
    train_token_types = []
    train_attention_mask = []
    train_y = []
    #index必須的
    for index , row in df.iterrows():
        content = row['full_content']
        content = clean_string(content)
        content_ckip_names = ast.literal_eval(row['ckip_names'])
        name_ans = ast.literal_eval(row['name'])
        #no ans
        if len(name_ans) == 0:
            continue

        for ckip_name in content_ckip_names:
            content_max_length = 512-3-len(ckip_name)
            if len(content) >= content_max_length:
                split_content = cut_sent(content)
                chunks = combine_sentence(split_content , content_max_length)
                for chunk in chunks:
                    if ckip_name not in chunk:
                        continue
                    input_ids = tokenizer.encode(ckip_name, chunk)
                    if(len(input_ids)>512):
                        continue
                    sep_index = input_ids.index(tokenizer.sep_token_id)
                    num_seg_a = sep_index + 1
                    num_seg_b = len(input_ids) - num_seg_a
                    segment_ids = [0]*num_seg_a + [1]*num_seg_b

                    input_mask = [1] * len(input_ids)

                    while len(input_ids) < 512:
                        input_ids.append(0)
                        input_mask.append(0)
                        segment_ids.append(0)

                    if ckip_name in name_ans:
                        train_y.append(1)
                    else:
                        train_y.append(0)

                    train_input_ids.append(input_ids)
                    train_token_types.append(segment_ids)
                    train_attention_mask.append(input_mask)
                    train_x.append((ckip_name,chunk)) 


            else:

                input_ids = tokenizer.encode(ckip_name, content)
                if(len(input_ids)>512):
                    continue
                sep_index = input_ids.index(tokenizer.sep_token_id)
                num_seg_a = sep_index + 1
                num_seg_b = len(input_ids) - num_seg_a
                segment_ids = [0]*num_seg_a + [1]*num_seg_b

                input_mask = [1] * len(input_ids)

                while len(input_ids) < 512:
                    input_ids.append(0)
                    input_mask.append(0)
                    segment_ids.append(0)


                if ckip_name in name_ans:
                    train_y.append(1)
                else:
                    train_y.append(0)

                train_input_ids.append(input_ids)
                train_token_types.append(segment_ids)
                train_attention_mask.append(input_mask)
                train_x.append((ckip_name,content)) 



    #全部跑完轉np
    train_input_ids = np.array(train_input_ids)
    train_token_types  = np.array(train_token_types)
    train_attention_mask = np.array(train_attention_mask)
    train_y = np.array(train_y)

    print(len(train_x))
    print(train_input_ids.shape)
    print(train_token_types.shape)
    print(train_attention_mask.shape)
    print(train_y.shape)
    
    return train_x , train_input_ids ,train_token_types  , train_attention_mask , train_y




In [5]:
train_x , train_input_ids ,train_token_types  , train_attention_mask , train_y  = get_model_input_split(train_df)
test_x , test_input_ids ,test_token_types  , test_attention_mask , test_y  = get_model_input_split(test_df)

3208
(3208, 512)
(3208, 512)
(3208, 512)
(3208,)
295
(295, 512)
(295, 512)
(295, 512)
(295,)


In [6]:
def eval(pred, ans):
    if bool(pred) is not bool(ans):
        return 0
    elif not pred and not ans:
        return 1
    else:
        pred = set(pred)
        ans = set(ans)
        interaction_len = len(pred & ans)
        if interaction_len == 0:
            return 0

        pred_len = len(pred)
        ans_len = len(ans)
        return 2 / (pred_len / interaction_len + ans_len / interaction_len)


def eval_all(pred_list, ans_list):
    assert len(pred_list) == len(ans_list)
    return sum(eval(p, a) for p, a in zip(pred_list, ans_list)) / len(pred_list)

In [7]:
class TrainDataset(Dataset):
    def __init__(self, input_ids , token_type_ids , attention_mask , y ):
        self.input_ids = input_ids
        self.token_type_ids = token_type_ids
        self.attention_mask = attention_mask
        self.y = y
    def __getitem__(self,idx):
        inputid = self.input_ids[idx]
        tokentype = self.token_type_ids[idx]
        attentionmask = self.attention_mask[idx]
        y = self.y[idx]

        return inputid , tokentype , attentionmask, y
    
    def __len__(self):
        return len(self.input_ids)
    

In [8]:
def get_eval(model,dataloader):
    model.eval()
    with torch.no_grad():
        total = 0
        correct = 0
        for data in dataloader:
            tokens_tensors, segments_tensors, masks_tensors, labels = [t.to(device) for t in data]
            outputs = model(input_ids=tokens_tensors, 
                                token_type_ids=segments_tensors, 
                                attention_mask=masks_tensors, 
                                labels=labels)
            pred = outputs[1]
            total += pred.size()[0]
            pred = torch.argmax(pred,dim=-1)
            correct += (pred==labels).sum().item()

    return correct/total 
    

In [49]:
BATCH_SIZE = 4
trainset = TrainDataset(train_input_ids ,train_token_types , train_attention_mask ,train_y)
trainloader = DataLoader(trainset, batch_size=BATCH_SIZE , shuffle=True)

testset = TrainDataset(test_input_ids ,test_token_types , test_attention_mask ,test_y)
testloader = DataLoader(testset, batch_size=BATCH_SIZE , shuffle=True)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device:", device)

NUM_LABELS = 2
tokenizer = BertTokenizer.from_pretrained(lm_path)
model  = BertForSequenceClassification.from_pretrained(lm_path,num_labels=NUM_LABELS)
model = model.to(device)
model.train()

optimizer = torch.optim.Adam(model.parameters(), lr=3e-6)

EPOCHS = 10
for epoch in range(EPOCHS):
    running_loss = 0.0
    total = 0
    correct = 0
    for data in trainloader:
        tokens_tensors, segments_tensors, masks_tensors, labels = [t.to(device) for t in data]
        optimizer.zero_grad()
        outputs = model(input_ids=tokens_tensors, 
                            token_type_ids=segments_tensors, 
                            attention_mask=masks_tensors, 
                            labels=labels)
        loss = outputs[0]
        pred = outputs[1]
        total += pred.size()[0]
        pred = torch.argmax(pred,dim=-1)
        correct += (pred==labels).sum().item()

        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        
    checkpoint_path = './QAModel/' + str(dataset) + '/' 
        
    torch.save(model.state_dict(),checkpoint_path + 'bert_wwm_QA_split_lr3e-6_' + str(epoch) + '.pkl')
    print('[epoch %d] loss: %.3f' %(epoch + 1, running_loss))
    print('train_acc:' ,correct/total)
    print('test_acc:' , get_eval(model,testloader))



device: cuda:0


- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[epoch 1] loss: 259.808
train_acc: 0.8668952618453866
test_acc: 0.9491525423728814
[epoch 2] loss: 110.264
train_acc: 0.955423940149626
test_acc: 0.8779661016949153
[epoch 3] loss: 58.655
train_acc: 0.9812967581047382
test_acc: 0.9491525423728814
[epoch 4] loss: 31.484
train_acc: 0.9918952618453866
test_acc: 0.9491525423728814
[epoch 5] loss: 25.537
train_acc: 0.9928304239401496
test_acc: 0.9457627118644067
[epoch 6] loss: 18.202
train_acc: 0.9962593516209476
test_acc: 0.9559322033898305
[epoch 7] loss: 14.457
train_acc: 0.996571072319202
test_acc: 0.9491525423728814
[epoch 8] loss: 22.214
train_acc: 0.9934538653366584
test_acc: 0.9389830508474576
[epoch 9] loss: 15.946
train_acc: 0.9943890274314214
test_acc: 0.9355932203389831
[epoch 10] loss: 11.066
train_acc: 0.9959476309226932
test_acc: 0.9559322033898305


In [10]:
def name_qa_binary_split(pred_name_list , news , checkpoint):
    class Testset(Dataset):
        def __init__(self, input_ids , token_type_ids , attention_mask , names):
            self.input_ids = input_ids
            self.token_type_ids = token_type_ids
            self.attention_mask = attention_mask
            self.names = names
        def __getitem__(self,idx):
            inputid = self.input_ids[idx]
            tokentype = self.token_type_ids[idx]
            attentionmask = self.attention_mask[idx]
            name = self.names[idx]
            return inputid , tokentype , attentionmask , name

        def __len__(self):
            return len(self.input_ids)
    
#     lm_path = './bert_wwm_pretrain_tbrain/'
    tokenizer = BertTokenizer.from_pretrained(lm_path)

    
    train_input_ids = []
    train_token_types = []
    train_attention_mask = []
    testing_name = []
    
    
    content = clean_string(news)
    
    max_length = 500
    
    split_content = cut_sent(content)
    chunks = combine_sentence(split_content , max_length)
    
    for chunk in chunks:
        for name in pred_name_list:
            if len(chunk) >= max_length:
                print('error !!!! lenth > 500')
                continue
            if name not in chunk:
                continue

            input_ids = tokenizer.encode(name, chunk)
            if(len(input_ids)>512):
                continue
            sep_index = input_ids.index(tokenizer.sep_token_id)
            num_seg_a = sep_index + 1
            num_seg_b = len(input_ids) - num_seg_a
            segment_ids = [0]*num_seg_a + [1]*num_seg_b

            input_mask = [1] * len(input_ids)

            while len(input_ids) < 512:
                input_ids.append(0)
                input_mask.append(0)
                segment_ids.append(0)

            train_input_ids.append(input_ids)
            train_token_types.append(segment_ids)
            train_attention_mask.append(input_mask)
            testing_name.append(name)
        
    train_input_ids = np.array(train_input_ids)
    train_token_types  = np.array(train_token_types)
    train_attention_mask = np.array(train_attention_mask)
    testing_name = np.array(testing_name)
    
    
    BATCH_SIZE = train_input_ids.shape[0]
    testset = Testset(train_input_ids ,train_token_types , train_attention_mask, testing_name)
    testloader = DataLoader(testset, batch_size=BATCH_SIZE)
    
    
    from transformers import BertForSequenceClassification
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    

    
    NUM_LABELS = 2
    tokenizer = BertTokenizer.from_pretrained(lm_path)
    model = BertForSequenceClassification.from_pretrained(lm_path,num_labels=NUM_LABELS)
    model.load_state_dict(torch.load(checkpoint))
    model = model.to(device)
    model.eval()


    with torch.no_grad():
        for data in testloader:
            tokens_tensors, segments_tensors, masks_tensors  = [t.to(device) for t in data[:-1]]
            name = data[-1]
            outputs = model(input_ids=tokens_tensors, 
                                token_type_ids=segments_tensors, 
                                attention_mask=masks_tensors)
            pred = torch.softmax(outputs[0] , dim = -1)
            pred = torch.argmax(pred,dim=-1)
            pred = pred.cpu().detach().numpy()
            pred_name_list = np.array(name)
            return list(pred_name_list[pred>0])

        

In [14]:
dataset = 2
path = './dataset/dataset' + str(dataset) + '/'

train_df = pd.read_csv(path + "tbrain_train.csv")
train_df = train_df.fillna('None')

test_df = pd.read_csv(path + "tbrain_test.csv")
test_df = test_df.fillna('None')

# train_df = pd.concat([train_df,test_df])
# lm_path = './bert_wwm_pretrain_tbrain/'
# lm_path = './pretrain_roberta_on_TBdata'
print(train_df.shape)
print(test_df.shape)


from transformers import BertForSequenceClassification
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device:", device)
print('dataset',dataset)

base_path = './QAModel/' + str(dataset) + '/'

split_checkpoint = base_path + 'roberta_QA_split2.pkl'

NUM_LABELS = 2
tokenizer = BertTokenizer.from_pretrained(lm_path)


# split model
split_model = BertForSequenceClassification.from_pretrained(lm_path,num_labels=NUM_LABELS)
split_model.load_state_dict(torch.load(split_checkpoint))
split_model = split_model.to(device)
split_model.eval()

split_pred = []

ans = []

for index,row in test_df.iterrows():
    news = row['full_content']
    ckip_names = ast.literal_eval(row['ckip_names'])
    names = ast.literal_eval(row['name'])
    
    if len(names) == 0 :
        continue
    
    
    split_result = name_qa_binary_split(ckip_names , news ,split_checkpoint)
    split_result = list(set(split_result))
    
    ans.append(names)
    split_pred.append(split_result)
print('score:',eval_all(split_pred,ans))
    

(4426, 4)
(491, 4)
device: cuda:0
dataset 2


- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


FileNotFoundError: [Errno 2] No such file or directory: './QAModel/2/roberta_QA_split2.pkl'

In [19]:
data = pd.read_csv('./tbrain/2020-07-29.csv')

correct =data[data['binary']==1]

correct
# correct = np.array(correct)

,article,binary,ckip_name,predict_name
0,檢調偵辦「三鑫集團」以投資俄羅斯賭場等名目吸金12億元案，發現三鑫集團負責人曾裕仁去年因債務...,1,"['陳男', '謝發布', '王妤昆', '曾裕仁']","['王妤昆', '曾裕仁']"
22,〔記者王善嬿／嘉市報導〕嘉義地檢署查獲台商王子南、羅則嬌等7人組集團進行兩岸地下匯兌，1年多...,1,"['王善嬿', '羅則嬌', '李瑋玲', '楊培芬', '吳仰孜', '鐘辛禾', '田某...","['羅則嬌', '李瑋玲', '楊培芬', '吳仰孜', '周志羽', '陳潔彥', '王子南']"
42,中華心蓮慈善助學協會理事長童行成(68歲)於4年前起，以協會名義推出「幼兒教育撲滿計畫」等集...,1,"['童行成', '鄧玉瑩']",['童行成']
48,漢唐光電科技董事長徐建志曾和和皮膚科名醫蔡佳雅發生不倫戀，他以不實財報佯稱漢唐公司將上市，販...,1,"['陳宏陽', '蔡佳雅', '徐雲其', '徐建志', '徐雲', '蔡夫']","['蔡佳雅', '徐雲其', '徐建志']"
89,檢調接獲報案，指位於台北市的台灣搜房公司2014年5月至12月間推出英國不動產投資方案，以保...,1,"['黃穎溪', '呂威東', '游仁汶', '吳珮', '高崇信', '彭傑', '高男']","['高崇信', '呂威東', '黃穎溪']"
125,國民黨宜蘭縣長參選人、羅東鎮長廖泓花遭控動用公所第二預備金補助其他鄉鎮活動涉嫌違反《預算法》...,1,"['羅東陳書豪', '涂文賢', '張鴻人', '廖泓花', '韓彥宏', '施仁旭']","['廖泓花', '韓彥宏']"
131,〔記者葉俞揚／新北報導〕怡婷科技公司正妹負責人曾怡婷以投資蘋果、三星等手機買賣，宣稱最高年報...,1,"['李文行', '怡婷', '張雅苓', '楊靖雯', '陳陽孜', '陳弘俊', '葉俞揚...","['曾雅琪', '陳弘俊', '曾怡婷', '楊靖雯']"
135,刑事局中部打擊犯罪中心破獲以林姓男子為首的「假轉帳、真詐騙」犯罪集團，專在網路佯裝買家，向賣...,1,"['陳男', '林昶杰', '王煌忠']",['林昶杰']
139,〔記者錢利忠／台北報導〕綽號「貢丸」的竹聯幫地隆會會長陳威富，涉嫌勾結運將江家瑋、林雲勳、梁...,1,"['陳威富', '江家瑋', '貢丸', '林雲勳', '錢利忠', '陳嫌', '梁姵寶']","['陳威富', '林雲勳', '梁姵寶', '江家瑋']"
146,屏東縣前車城鄉長林俊豪於在2010年任職鄉長後利用招標機會收取不法回扣，經去年屏東地院重判有...,1,"['賴筱涵', '李建冰', '賴家賢', '吳宛財', '林俊', '黃進郁', '楊白萱...","['李建冰', '吳宛財', '黃進郁', '林俊豪', '蕭信宏', '蔡佳雅', '王品..."


In [61]:
data = pd.read_csv('./tbrain/2020-07-29_test.csv')

test_df =data[data['binary']==1]
# test_df = pd.read_csv("./dataset/tbrain_test (2).csv")
# test_df = test_df.fillna('None')

names =  test_df['predict_name'].tolist()
contents = np.array(test_df['article'].tolist())
ckip_names = test_df['ckip_name'].tolist()

valid_count = 0
score = 0.0

pred_list = []
ans_list = []

for i in range(len(contents)):
    content = contents[i]
    content = clean_string(content)
    content_ckip_names = ast.literal_eval(ckip_names[i])
    name_list = ast.literal_eval(names[i])
    
    
    if len(name_list) == 0:
        continue
        
    pred =  check_pred_name_is_real_ans(content_ckip_names,content)
    
    print('---------------')
    print('pred : ' , pred)
    print('ans : ', name_list)
        
        
    pred_list.append(pred)
    ans_list.append(name_list)
    
eval_all(pred_list,ans_list)
    
   
    
    
    
    
    



- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


---------------
pred :  ['王妤昆', '曾裕仁']
ans :  ['王妤昆', '曾裕仁']


- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


---------------
pred :  ['羅則嬌', '李瑋玲', '楊培芬', '吳仰孜', '鐘辛禾', '周志羽', '陳潔彥', '王子南']
ans :  ['羅則嬌', '李瑋玲', '楊培芬', '吳仰孜', '周志羽', '陳潔彥', '王子南', '鐘辛禾']


- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


---------------
pred :  ['吳怡易']
ans :  ['吳怡易']


- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


---------------
pred :  ['連庭亨', '毛維倫']
ans :  ['連庭亨']


0.9323232323232324

In [60]:
def check_pred_name_is_real_ans(pred_name_list,news):
    class Testset(Dataset):
        def __init__(self, input_ids , token_type_ids , attention_mask):
            self.input_ids = input_ids
            self.token_type_ids = token_type_ids
            self.attention_mask = attention_mask
        def __getitem__(self,idx):
            inputid = self.input_ids[idx]
            tokentype = self.token_type_ids[idx]
            attentionmask = self.attention_mask[idx]

            return inputid , tokentype , attentionmask

        def __len__(self):
            return len(self.input_ids)
    
    tokenizer = BertTokenizer.from_pretrained('./pretrain_roberta_on_TBdata/')

    content = clean_string(news)
    train_input_ids = []
    train_token_types = []
    train_attention_mask = []
        
    for name in pred_name_list:
        
        content_max_length = 512-3-len(name)
        
        if len(content) >= content_max_length:
            content = content[:content_max_length]
            
        input_ids = tokenizer.encode(name, content)
        if(len(input_ids)>512):
            continue
        sep_index = input_ids.index(tokenizer.sep_token_id)
        num_seg_a = sep_index + 1
        num_seg_b = len(input_ids) - num_seg_a
        segment_ids = [0]*num_seg_a + [1]*num_seg_b

        input_mask = [1] * len(input_ids)

        while len(input_ids) < 512:
            input_ids.append(0)
            input_mask.append(0)
            segment_ids.append(0)
            
        train_input_ids.append(input_ids)
        train_token_types.append(segment_ids)
        train_attention_mask.append(input_mask)
        
    train_input_ids = np.array(train_input_ids)
    train_token_types  = np.array(train_token_types)
    train_attention_mask = np.array(train_attention_mask)
    
    
    BATCH_SIZE = train_input_ids.shape[0]
    
    testset = Testset(train_input_ids ,train_token_types , train_attention_mask)
    testloader = DataLoader(testset, batch_size=BATCH_SIZE)
    
    
    from transformers import BertForSequenceClassification
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#     print("device:", device)
#     checkpoint_path = './QAModel/1/roberta_QA_split_lr3e-6_4.pkl' 
        
#     torch.save(model.state_dict(),checkpoint_path + 'roberta_QA_split_lr3e-6_' + str(epoch) + '.pkl')
#     lm_path = './QAModel/1/roberta_QA_split_lr3e-6_4.pkl' 
    NUM_LABELS = 2
    tokenizer = BertTokenizer.from_pretrained('./pretrain_roberta_on_TBdata/')
    model = BertForSequenceClassification.from_pretrained('./pretrain_roberta_on_TBdata/',num_labels=NUM_LABELS)
    check_point = './QAModel/1/bert_wwm_QA_split_lr3e-6_9.pkl' 
    model.load_state_dict(torch.load(check_point))
    model = model.to(device)
    model.eval()


    with torch.no_grad():
        for data in testloader:
            tokens_tensors, segments_tensors, masks_tensors = [t.to(device) for t in data]
            outputs = model(input_ids=tokens_tensors, 
                                token_type_ids=segments_tensors, 
                                attention_mask=masks_tensors)
            pred = torch.softmax(outputs[0] , dim = -1)
            torch.set_printoptions(precision=10)
#             print(pred)
            pred = torch.argmax(pred,dim=-1)
            pred = pred.cpu().detach().numpy()
            pred_name_list = np.array(pred_name_list)
            return list(pred_name_list[pred>0])
        

    



    
    